

```
This is practice note book to practice deep learning with pytorch
### Source: https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
```



In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Downloading training data form open datasets
training_data= datasets.FashionMNIST(
    root= 'data',
    train= True,
    download=True,
    transform= ToTensor(),

)


  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
# downloading test dataset form open datasets

test_data= datasets.FashionMNIST(
    root= 'data',
    train= False,
    download=True,
    transform= ToTensor(),


)

Creating a data loader which wraps an iteratable over dataset and support automatic batching,sampling,shuffling and multiprocess batch loading

In [4]:
batch_size= 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size= batch_size)
test_dataloader= DataLoader(test_data, batch_size= batch_size)


for X, y in test_dataloader:
  print(f"Shape of X [N,C,H,W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break




Shape of X [N,C,H,W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


Creating model

In [9]:
# Get cpu or gpu for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

#Defining model

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack= nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10),
    )

  def forward (self,x):
    x= self.flatten(x)
    logits= self.linear_relu_stack(x)
    return logits

model= NeuralNetwork().to(device)
print(model)


Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Optimizing the model parameters


In [10]:
loss_fn= nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr= 1e-3)

In [16]:
def train(dataloader, model,loss_fn,optimizer):
  size= len(dataloader.dataset)
  model.train()
  for batch, (X,y) in enumerate(dataloader):
    X,y = X.to(device), y.to(device)

    #computing prediction error
    pred= model(X)
    loss= loss_fn(pred,y)

    #Backpropagation

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 ==0:
      loss,current = loss.item(), batch*len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")






In [21]:
def test(dataloader, model,loss_fn):
  size= len(dataloader.dataset)
  num_batches= len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X,y in dataloader:
      X,y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred,y).item()
      correct += (pred.argmax(1)==y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test error: \n Accuracy: {(100*correct):> 0.1f}%, Avg loss: {test_loss:>8f} \n ")

In [22]:
epochs= 5
for t in range(epochs):
  print(f"Epoch {t+1}\n-----------------------")
  train(train_dataloader,model,loss_fn, optimizer)
  test(test_dataloader, model,loss_fn)

print("Done!")


Epoch 1
-----------------------
loss: 1.891699 [    0/60000]
loss: 1.853093 [ 6400/60000]
loss: 1.743108 [12800/60000]
loss: 1.782581 [19200/60000]
loss: 1.659221 [25600/60000]
loss: 1.635518 [32000/60000]
loss: 1.645719 [38400/60000]
loss: 1.552070 [44800/60000]
loss: 1.583394 [51200/60000]
loss: 1.475168 [57600/60000]
Test error: 
 Accuracy:  59.8%, Avg loss: 1.495933 
 
Epoch 2
-----------------------
loss: 1.568562 [    0/60000]
loss: 1.526188 [ 6400/60000]
loss: 1.392561 [12800/60000]
loss: 1.456638 [19200/60000]
loss: 1.325453 [25600/60000]
loss: 1.351734 [32000/60000]
loss: 1.348142 [38400/60000]
loss: 1.276132 [44800/60000]
loss: 1.315132 [51200/60000]
loss: 1.217212 [57600/60000]
Test error: 
 Accuracy:  62.7%, Avg loss: 1.241560 
 
Epoch 3
-----------------------
loss: 1.323166 [    0/60000]
loss: 1.297219 [ 6400/60000]
loss: 1.147109 [12800/60000]
loss: 1.245293 [19200/60000]
loss: 1.109234 [25600/60000]
loss: 1.159279 [32000/60000]
loss: 1.165011 [38400/60000]
loss: 1.10005

Saving the model

In [23]:
torch.save(model.state_dict(),"model.pth")
print("Saved PyTorch Model state to model.pth")

Saved PyTorch Model state to model.pth


Loading Model

In [24]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

Making prediction

In [27]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",  
           ]
model.eval()
x,y = test_data[3][0], test_data[3][1]
with torch.no_grad():
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)],classes[y]
  print(f"Predicted: {predicted} , Actual: {actual} ")

Predicted: Trouser , Actual: Trouser 
